In [1]:
# Load text from Project Gutenberg URL
import urllib
from urllib.request import urlopen
url_template = 'http://www.gutenberg.org/cache/epub/1661/pg1661.txt'
f = urllib.request.urlopen(url_template)
text = f.read().decode("utf-8") 
f.close()


In [2]:
#Pre- Processing Part
import re
from nltk.tokenize import sent_tokenize
# Preprocessing part
text=text.replace('\n', ' ')
text=text.replace('"s', ' ')
text=text.replace(',', ' ')
text=text.replace('/s', ' ')
text=text.replace('"', ' ')
text=text.replace ("'", '')
text = text.lower()
# text= re.sub('[(){}<>] ', ' ', text)

sent_tokenize_list = sent_tokenize(text)
# print(len(sent_tokenize_list))
sent_start_end = []
words = []

# print(sent_tokenize_list)

for sent in sent_tokenize_list:
    sent = ' '.join(('<s>',sent,'</s>'))
    sent = sent.replace('!', ' ')
    sent = sent.replace('.', ' ')
    sent = sent.replace('?', ' ')
    sent = sent.replace('-', ' ')
    sent = sent.replace('ï»¿', ' ')
    sent = sent.replace('*', ' ')   
    sent = sent.replace('\n', ' ') 
    sent = sent.replace(':', ' ') 
    sent = sent.replace(';', ' ')    
    
#     sent = str(TextBlob(sent).correct())          
    sent = re.sub( '\s{2,}', ' ', sent).strip()    #Trying to remove extra spaces from sent
#     print(sent)
    for word in sent.split(' '):
        words.append(word)     #Words List
    sent_start_end.append(sent)  
# print(sent_start_end[500:1000])

In [3]:
# print(words [500:1000])
print("Total Tokens = ", len(words))

Total Tokens =  122392


In [4]:
from sklearn.model_selection import train_test_split
# Splitting the file into 80% and 20%
train, test = train_test_split(sent_start_end, test_size = 0.2)


In [5]:
word_train = []
for sent in train:
    for word in sent.split(' '):
        word_train.append(word)                  #Training set words list
print("Training token count" , len(word_train))
# print(word_train[69000:70000])


Training token count 98514


In [6]:
word_test = []
for sent in test:
    for word in sent.split(' '):
        word_test.append(word)                  #Testing set words list
print(" Testing token count", len(word_test))
# print(word_test)


 Testing token count 23878


In [7]:
from collections import Counter
unigram_count=Counter(word_train)

#MLE Probabilities of unigram
sum2 = 0
for key in unigram_count:
    sum2 = float(sum2 + unigram_count.get(key))

unigram_prob = {}
for key in unigram_count:
    unigram_prob[key] = round( unigram_count.get(key) / sum2, 6)

#Displaying the results for unigrams
print("Total training token Count (seen) =" , len(word_train))
# print("Distict Tokens = ", len(set(word_train)))
print("Distinct Unigrams count =", len(unigram_count))
print("Expected Unigrams Count = ", len(unigram_count)   )
gram1 = sorted(unigram_prob.items(), key=lambda count: count[1], reverse =True)
print("Top 10 MLE for unigram")
print (gram1[:10])


Total training token Count (seen) = 98514
Distinct Unigrams count = 7569
Expected Unigrams Count =  7569
Top 10 MLE for unigram
[('<s>', 0.05585), ('</s>', 0.05585), ('the', 0.047709), ('and', 0.02555), ('i', 0.024646), ('to', 0.023073), ('of', 0.022454), ('a', 0.02223), ('in', 0.014983), ('that', 0.014303)]


In [8]:
####Bigram tokens

biwords = []
for index, item in enumerate(word_train):
    if index == len(word_train)-1:
        break
    biwords.append(item+' '+word_train[index+1])
# print(biwords)
bigram_count = Counter(biwords)

# Bigram Probabilities with and without smoothing
bigram_prob = {}
for key in bigram_count:
    uniword = key.split(' ')
#     print(uniword[0])
#     print(key)
    if (bigram_count[key] == 0):
        bigram_prob[key] =0
        bigram_prob_smooth[key] = 0
    else:
        bigram_prob[key] = round( bigram_count.get(key) / float(unigram_count.get(uniword[0])), 6)
print("Corpus Training bigram count (seen) =", sum(list(bigram_count.values())))
print("Corpus Training bigram count (distinct) =", len(bigram_count))
print("Expected Training bigram Count (distinct) = ", (len(unigram_count) * len(unigram_count)))  

#displaying Probabilities for bigram
gram2 = sorted(bigram_prob.items(), key=lambda count: count[1], reverse =False)
print("Top 10 MLE for bigram")
print (gram2[:10])


Corpus Training bigram count (seen) = 98513
Corpus Training bigram count (distinct) = 43467
Expected Training bigram Count (distinct) =  57289761
Top 10 MLE for bigram
[('<s> start', 0.000182), ('<s> jephro', 0.000182), ('<s> owe', 0.000182), ('<s> visited', 0.000182), ('<s> recently', 0.000182), ('<s> threatens', 0.000182), ('<s> youd', 0.000182), ('<s> evidence', 0.000182), ('<s> vincent', 0.000182), ('<s> coarse', 0.000182)]


In [9]:
####Trigram Tokens
triwords = []
for index, item in enumerate(word_train):
    if index == len(word_train)-2:
        break
    triwords.append(item+' '+word_train[index+1]+' '+word_train[index+2])
trigram_count = Counter(triwords)

trigram_prob = {}
for key in trigram_count:
    biword = key.split(' ')
    trigram_prob[key] = round( trigram_count.get(key) / bigram_count.get(biword[0] +' ' + biword[1]), 6)


print("Corpus Training trigram count (seen) =", sum(list(trigram_count.values())))
print("Corpus Training trigram count (distinct) =", len(trigram_count))
print("Training Expected trigrams Count (distinct) = ", len(unigram_count) * len(unigram_count) * len(unigram_count)   )

gram3 = sorted(trigram_prob.items(), key=lambda count: count[1], reverse =False)
print("Least 10 MLE for trigram")
print (gram3[:10])


Corpus Training trigram count (seen) = 98512
Corpus Training trigram count (distinct) = 74686
Training Expected trigrams Count (distinct) =  433626201009
Least 10 MLE for trigram
[('</s> <s> start', 0.000182), ('</s> <s> jephro', 0.000182), ('</s> <s> owe', 0.000182), ('</s> <s> visited', 0.000182), ('</s> <s> recently', 0.000182), ('</s> <s> threatens', 0.000182), ('</s> <s> youd', 0.000182), ('</s> <s> evidence', 0.000182), ('</s> <s> vincent', 0.000182), ('</s> <s> coarse', 0.000182)]


In [10]:
####Quadrigram Tokens

quadriwords = []
for index, item in enumerate(word_train):
    if index == len(word_train)-3:
        break
    quadriwords.append(item+' '+word_train[index+1]+' '+word_train[index+2]+' '+word_train[index+3])
quadrigram_count = Counter(quadriwords)
# print(quadrigram_count)

quadrigram_prob = {}
for key in quadrigram_count:
    triword = key.split(' ')
#     print(triword[0:3])
#     print(key)
    quadrigram_prob[key] = round( quadrigram_count.get(key) / trigram_count.get(triword[0] +' ' + triword[1]+' ' + triword[2]), 6)
# print(quadrigram_prob)

print("Corpus Training quadrigram count (seen) =", sum(list(quadrigram_count.values())))
print("Corpus Training quadrigram count (distinct) =", len(quadrigram_count))
print("Expected quadrigrams Count (distinct) = ", len(unigram_count) * len(unigram_count) * len(unigram_count)  * len(unigram_count)  )

gram4 = sorted(quadrigram_prob.items(), key=lambda count: count[1], reverse =False)
print("Least 10 MLE for quadrigram")
print (gram4[:10])


Corpus Training quadrigram count (seen) = 98511
Corpus Training quadrigram count (distinct) = 89722
Expected quadrigrams Count (distinct) =  3282116715437121
Least 10 MLE for quadrigram
[('</s> <s> i fully', 0.001247), ('</s> <s> i guessed', 0.001247), ('</s> <s> i warn', 0.001247), ('</s> <s> i might', 0.001247), ('</s> <s> i advertised', 0.001247), ('</s> <s> i seated', 0.001247), ('</s> <s> i assured', 0.001247), ('</s> <s> i wouldnt', 0.001247), ('</s> <s> i yelled', 0.001247), ('</s> <s> i braved', 0.001247)]


In [11]:
####Pentigram Tokens

pentiwords = []
for index, item in enumerate(word_train):
    if index == len(word_train)-4:
        break
    pentiwords.append(item+' '+word_train[index+1]+' '+word_train[index+2]+' '+word_train[index+3]+' '+word_train[index+4])
pentigram_count = Counter(pentiwords)
# print(pentigram_count)

pentigram_prob = {}
for key in pentigram_count:
    quadriword = key.split(' ')
#     print(quadriword[0:4])
#     print(key)
    pentigram_prob[key] = round( pentigram_count.get(key) / quadrigram_count.get(quadriword[0] +' ' + quadriword[1]+' ' + quadriword[2]+' ' + quadriword[3]), 6)
# print(pentigram_prob)

print("Corpus Training pentigram count (seen) =", sum(list(pentigram_count.values())))
print("Corpus Training pentigram count (distinct) =", len(pentigram_count))
print("Expected pentigrams Count = ", len(unigram_count) * len(unigram_count) * len(unigram_count)  * len(unigram_count) * len(unigram_count)  )

gram5 = sorted(pentigram_prob.items(), key=lambda count: count[1], reverse =False)
print("Least 10 MLE for pentigram")
print (gram5[:10])



Corpus Training pentigram count (seen) = 98510
Corpus Training pentigram count (distinct) = 95769
Expected pentigrams Count =  24842341419143568849
Least 10 MLE for pentigram
[('</s> <s> it is past', 0.007752), ('</s> <s> it is said', 0.007752), ('</s> <s> it is high', 0.007752), ('</s> <s> it is just', 0.007752), ('</s> <s> it is well', 0.007752), ('</s> <s> it is due', 0.007752), ('</s> <s> it is wednesday', 0.007752), ('</s> <s> it is one', 0.007752), ('</s> <s> it is really', 0.007752), ('</s> <s> it is fear', 0.007752)]


In [12]:
#Building nested dictionary 
import random
unigram_dictt = {}
bigram_dictt = {}
trigram_dictt = {}
quadrigram_dictt = {}
pentigram_dictt = {}
def build_dictionary(model):
    if model == "unigram":
#         print("Unigram Dictionary")
        for i in range(len(words)): 
            if not unigram_dictt.get(words[i]): 
                unigram_dictt[words[i]] = 1 
            else: 
                unigram_dictt[words[i]] += 1 
        summ = sum (unigram_dictt.values())

        for key in unigram_dictt:
            prob =round(unigram_dictt.get(key) / summ,6)
            unigram_dictt[key] = prob


        gram1_iter= iter(unigram_dictt)

        
    if model == "bigram":
#         print("bigram dictionary")
        for i in range(len(words) - 1): 
            if not bigram_dictt.get(words[i]): 
                bigram_dictt[words[i]] = {} 
            if not bigram_dictt[words[i]].get(words[i+1]): 
                bigram_dictt[words[i]][words[i+1]] = 1 
            else: 
                bigram_dictt[words[i]][words[i+1]] += 1 
        for outer_key in bigram_dictt:
            subdict = dict(bigram_dictt[outer_key])
            summ = sum (subdict.values())
            for inner_key in subdict: 
                prob =round(subdict.get(inner_key)/ summ,6)
                bigram_dictt[outer_key][inner_key] = prob
        gram2_iter= iter(bigram_dictt)


    elif model == "trigram":  
#         print("trigram ditionary")
        for i in range(len(words) - 2): 
            if not trigram_dictt.get(words[i] + ' ' + words[i+1]): 
                trigram_dictt[words[i] + ' ' + words[i+1]] = {} 
            if not trigram_dictt[words[i] + ' ' + words[i+1]].get(words[i+2]): 
                trigram_dictt[words[i] + ' ' + words[i+1]][words[i+2]] = 1 
            else: 
                trigram_dictt[words[i] + ' ' + words[i+1]][words[i+2]] += 1 
    
        for outer_key in trigram_dictt:
            subdict = dict(trigram_dictt[outer_key])
            summ = sum (subdict.values())
            for inner_key in subdict: 
                prob =round(subdict.get(inner_key)/ summ,6)
                trigram_dictt[outer_key][inner_key] = prob
                
    
        gram3_iter= iter(trigram_dictt)

        
    elif model == "quadrigram":
#         print("quadrigram dictionary")
        for i in range(len(words) - 3): 
            if not quadrigram_dictt.get(words[i] + ' ' + words[i+1] + ' ' + words [i+2]): 
                quadrigram_dictt[words[i] + ' ' + words[i+1]+ ' ' + words[i+2]] = {} 
            if not quadrigram_dictt[words[i] + ' ' + words[i+1] +' ' + words[i+2]].get(words[i+3]): 
                quadrigram_dictt[words[i] + ' ' + words[i+1] + ' ' + words[i+2]][words[i+3]] = 1 
            else: 
                quadrigram_dictt[words[i] + ' ' + words[i+1] + ' ' + words[i+2]][words[i+3]] += 1 
        for outer_key in quadrigram_dictt:
            subdict = dict(quadrigram_dictt[outer_key])
            summ = sum (subdict.values())
            for inner_key in subdict: 
                prob =round(subdict.get(inner_key)/ summ,6)
                quadrigram_dictt[outer_key][inner_key] = prob
        gram4_iter= iter(quadrigram_dictt)


    elif model == "pentigram":
#         print("pentigram dictionary")
        for i in range(len(words) - 4): 
            if not pentigram_dictt.get(words[i] + ' ' + words[i+1] + ' ' + words [i+2]+ ' ' + words [i+3]): 
                pentigram_dictt[words[i] + ' ' + words[i+1]+ ' ' + words[i+2]+ ' ' + words [i+3]] = {} 
            if not pentigram_dictt[words[i] + ' ' + words[i+1] +' ' + words[i+2]+ ' ' + words [i+3]].get(words[i+4]): 
                pentigram_dictt[words[i] + ' ' + words[i+1] + ' ' + words[i+2]+ ' ' + words [i+3]][words[i+4]] = 1 
            else: 
                pentigram_dictt[words[i] + ' ' + words[i+1] + ' ' + words[i+2]+ ' ' + words [i+3]][words[i+4]] += 1 
        for outer_key in pentigram_dictt:
            subdict = dict(pentigram_dictt[outer_key])
            summ = sum (subdict.values())
            for inner_key in subdict: 
                prob =round(subdict.get(inner_key)/ summ,6)
                pentigram_dictt[outer_key][inner_key] = prob
        gram5_iter= iter(pentigram_dictt)




In [13]:
import numpy as np
def Gen_Sentence(model_name):
    print("*********************Generating Random sentences based on", model_name, "***********************\n")
    if (model_name == "unigram"):
        for i in range(0,8):
            new = ""
            sen_gen = '<s>'
            while (new != "</s>" and len(sen_gen.split(' ')) < 30):
                subdict = unigram_dictt
                if subdict:
                    listt = np.random.multinomial(20, list(subdict.values()),1)
                    new= random.choice(list(subdict))
                sen_gen = sen_gen + " " + new
            sen_gen =sen_gen +'</s>'
            print (sen_gen, "\n")
            
    if model_name == "bigram":
        for i in range(0,8):
            prev_word ="<s>" 
            new = ""
            sen_gen = prev_word
            while (new != "</s>" and len(sen_gen.split(' ')) < 30):
                subdict = bigram_dictt.get(prev_word)
                if subdict:
                    new= random.choice(list(subdict))
                prev_word = new
                sen_gen = sen_gen + " " + new
            sen_gen =sen_gen +'</s>'
            print (sen_gen, "\n")

        
    if model_name == "trigram":
         for i in range(0,8):
            temp='<s>'
            new =  random.choice(list(bigram_dictt.get(temp)))
            prev_word = temp + " " + new
            sen_gen = prev_word
            while (new != "</s>" and len(sen_gen.split(' ')) < 30):
                subdict = trigram_dictt.get(prev_word)
                if subdict:
                    temp = new
                    listt = np.random.multinomial(20, list(subdict.values()),1)
                    new= random.choice(list(subdict))
                sen_gen = sen_gen + " " + new + " " 
                prev_word = temp + " " + new
            sen_gen =sen_gen +'</s>'
            print (sen_gen, "\n")
    if model_name == "quadrigram":
        for i in range(0,8):
            temp1='<s>'
            temp2 =  random.choice(list(bigram_dictt.get(temp1)))
            new =  random.choice(list(trigram_dictt.get(temp1 + " " + temp2)))
            prev_word = temp1 + " " + temp2 + " " + new
            sen_gen = prev_word
            while (new != "</s>" and len(sen_gen.split(' ')) < 30):
                subdict = quadrigram_dictt.get(prev_word)
                if subdict:
                    temp1 = temp2
                    temp2 = new
                    listt = np.random.multinomial(20, list(subdict.values()),1)
                    new= random.choice(list(subdict))
                sen_gen = sen_gen + " " + new + " " 
                prev_word = temp1 +  " " + temp2 + " " + new
            sen_gen =sen_gen +'</s>'
            print (sen_gen, "\n")
    if model_name == "pentigram":
        for i in range(0,8):
            temp1='<s>'
            temp2 =  random.choice(list(bigram_dictt.get(temp1)))
            temp3 =  random.choice(list(trigram_dictt.get(temp1 + " " + temp2)))
            new =   random.choice(list(quadrigram_dictt.get(temp1 + " " + temp2 + " " + temp3)))
            prev_word = temp1 + " " + temp2 + " " +temp3 + " "+ new
            sen_gen = prev_word
            while (new != "</s>" and len(sen_gen.split(' ')) < 30):
                subdict = pentigram_dictt.get(prev_word)
                if subdict:
                    temp1 = temp2
                    temp2 = temp3
                    temp3 = new
                    listt = np.random.multinomial(20, list(subdict.values()),1)
                    new= random.choice(list(subdict))
                sen_gen = sen_gen + " " + new + " " 
                prev_word = temp1 +  " " + temp2 + " " + temp3 +" " + new
            sen_gen =sen_gen +'</s>'
            print (sen_gen, "\n")

        
model = [ 'unigram', 'bigram', 'trigram', 'quadrigram', 'pentigram']
for modd in model:
    build_dictionary(modd)
    Gen_Sentence (modd)
        
    

*********************Generating Random sentences based on unigram ***********************

<s> oclock audible donna refers visitors hole rather office veiled settling supposed officials wheeler foreman influence strand natured carved delirium serpent places served escort surly fiery reproach shelf supporting boat</s> 

<s> valet smiling waiter any) dated perform too union specimen rubber absence peaked heroic generations regretted 4th maintenance waistcoat temper frowning calf look troubled pencil horrid fits conception unmistakable requests</s> 

<s> conclusion dated originator entered 6d stark madam started balustraded crossed faith edge discretion occupations mostly foretold prosecution txt jump affecting noised motives steep hardy feminine erroneous policeman ostlers petulance</s> 

<s> silvered activity sovereigns league sun mile pit labyrinth san allusion tied brims deepest thread demeanour peasant constraint 18 arresting cab concentrate friends road addressing seconds parcel ren

In [51]:
#Caluclating d values for good turing

countOfCounts = {}
gd_count= {}
def countOfCountsTable():
    

    count_list =  set(bigram_count.values())
    countsOfCounts = {}
    for c in count_list :
        countOfCounts[c] = 0
        for key, keycount in bigram_count.items():
            if keycount == c:
                countOfCounts[c] += 1
    return countOfCounts


countOfCounts = countOfCountsTable()
N=sum(list(bigram_count.values()))
dist = len(bigram_count.values())
N_0 = (len(unigram_count) * len(unigram_count)) - dist
# print("Bigrams seen =", N)
# print("distinct Bigrams", dist)
# print("Bigrams not seen", N_0)


gd_count [0] = countOfCounts[1] / N
countOfCounts[0] =N_0
d =0 
for i in range(1,10):     #max(countOfCounts)
    gd_count[i] = ((i+1) * countOfCounts[i+1]) /countOfCounts [i]
    
# print("Updated count values of bigrams corresponding to each original count is mentioned in key value pair")

for i in range (1,7):
    d = d +(i - gd_count[i])
#     print(i, gd_count[i])
d=  round(d/6, 2)
print("Observed value of d for good turing=", d)

Observed value of d for good turing= 0.9


In [34]:
# Calculating  Smooth probabilities Add1 and Good Turing
bigram_prob_smooth = {}
bigram_count_smooth = {}
bigram_prob_gd = {}
bigram_count_gd = {}


for key in bigram_count:
    uniword = key.split(' ')
#     print(uniword[0])
#     print(key)
        
        ######Calculating Add 1 Smoothed count and probability#############
        
    smooth = (bigram_count.get(key) + 1) / (unigram_count.get(uniword[0]) + (len(unigram_count)))   
    #Adding one to each word and dividing by vocaulary size for updating the count
    bigram_prob_smooth[key] = round (smooth, 6)     #Updated probaility
    bigram_count_smooth[key] = bigram_prob_smooth[key] * unigram_count[uniword[0]]    #Updated Add 1 smooth count
    ######Calculating Good Turing Smoothed count and probability#############
        
    bigram_count_gd[key] = bigram_count[key] - d    #Using observed d value for calculating updated gd count
        
    bigram_prob_gd[key] = ( bigram_count_gd[key] / unigram_count[uniword[0]])   #Calulating probability with updated mle



In [39]:
# Calculating Probability of a given sentence with some specific model
import math as calc

def probability(sentence, model_name):
    P= 0
    sent_words = sentence.lower().split(' ')
    if model_name == 'unigram':
      
        for item in sent_words:
            if item in unigram_prob:
                P = P + calc.log(unigram_prob[item]) 
        return P
    
    elif model_name == 'bigram':
        sent_biword = []
        for index, item in enumerate(sent_words):
            if index == len(sent_words)-1:
                break
            sent_biword.append(item+' '+sent_words[index+1])
        for item in sent_biword:
            if item in bigram_prob:
                P = P +calc.log(bigram_prob[item])
        return P
    
    elif model_name == 'add1':
        sent_biword = []
        for index, item in enumerate(sent_words):
            if index == len(sent_words)-1:
                break
            sent_biword.append(item+' '+sent_words[index+1])
        for item in sent_biword:
            if item in bigram_prob_smooth:
                P = P +calc.log(bigram_prob_smooth[item])
        return P
    elif model_name == 'gd':
        sent_biword = []
        for index, item in enumerate(sent_words):
            if index == len(sent_words)-1:
                break
            sent_biword.append(item+' '+sent_words[index+1])
        for item in sent_biword:
            if item in bigram_prob_gd:
#                 print(item, bigram_prob[item])
                P = P +calc.log(bigram_prob_gd[item])
        return P
    elif model_name == 'trigram':
        sent_triword = []
        for index, item in enumerate(sent_words):
            if index == len(sent_words)-2:
                break
            sent_triword.append(item+' '+sent_words[index+1]+' '+sent_words[index+2])
        for item in sent_triword:
            if item in trigram_prob:
                P = P + calc.log(trigram_prob[item])
        return P
    elif model_name == 'quadrigram':
        sent_quadriword = []
        for index, item in enumerate(sent_words):
            if index == len(sent_words)-3:
                break
            sent_quadriword.append(item+' '+sent_words[index+1]+' '+sent_words[index+2]+' '+sent_words[index+3])
        for item in sent_quadriword:
            if item in quadrigram_prob:
                P = P + calc.log(quadrigram_prob[item])
        return P
    elif model_name == 'pentigram':
        sent_pentiword = []
        for index, item in enumerate(sent_words):
            if index == len(sent_words)-4:
                break
            sent_pentiword.append(item+' '+sent_words[index+1]+' '+sent_words[index+2]+' '+sent_words[index+3]+' '+sent_words[index+4])
        for item in sent_pentiword:
            if item in pentigram_prob:
                P = P + calc.log(pentigram_prob[item])
        return P
    
    
from random import randint
sent = test[randint(0, len(test)-1)]    # Randomly picking the sentence from test data set
print ("Random sentence from thetest dataset =", sent)
P  = probability(sent, "unigram")
print("Probability with unigram, = (Unsmoothed)"  , np.exp(P))
P = probability(sent, "bigram")
print("Probability with bigram, = (Unsmoothed)"  ,np.exp(P))
P  = probability(sent, "trigram")
print("Probability with trigram, = (Unsmoothed)"  , np.exp(P))
P  =probability(sent, "quadrigram")
print("Probability with quadrigram, = (Unsmoothed)"  , np.exp(P))
P  =probability(sent, "pentigram")
print("Probability with pentigram, = (Unsmoothed)"  , np.exp(P))
    

Random sentence from thetest dataset = <s> you have called me names enough said he i will not stand it any longer </s>
Probability with unigram, = (Unsmoothed) 4.78815597392e-45
Probability with bigram, = (Unsmoothed) 2.41264062984e-17
Probability with trigram, = (Unsmoothed) 0.00049785744825
Probability with quadrigram, = (Unsmoothed) 1.0
Probability with pentigram, = (Unsmoothed) 1.0


In [40]:
import random
for i in range (0, 10):
    key =random.choice(list(bigram_count.keys()))
    print("Unsmoothed Count '", key, "' = ", bigram_count[key])
    print ("Smoothed Count '", key, "'=",  bigram_count_smooth[key])
print("As we can see, there is huge reduction in the number of counts of particular bigrams, When we do Add-one Smoothing.\
This happens because we have distributed the probability mass of existing bigrams to the bigrams which do not exist in the\
corpus. So we have to reduce the probability of already existing bigrams and hence the their count gets decreased, but, so \
huge reduction of count is not desirable. Therefore Add 1 smoothing is not a good solution for language modelling. ")

Unsmoothed Count ' say is ' =  1
Smoothed Count ' say is '= 0.021141
Unsmoothed Count ' snuff then ' =  1
Smoothed Count ' snuff then '= 0.001056
Unsmoothed Count ' the creases ' =  1
Smoothed Count ' the creases '= 0.7661
Unsmoothed Count ' winchester to ' =  1
Smoothed Count ' winchester to '= 0.002112
Unsmoothed Count ' lee a ' =  1
Smoothed Count ' lee a '= 0.0015840000000000001
Unsmoothed Count ' now made ' =  1
Smoothed Count ' now made '= 0.046439999999999995
Unsmoothed Count ' away </s> ' =  6
Smoothed Count ' away </s> '= 0.07686
Unsmoothed Count ' even gaunter ' =  1
Smoothed Count ' even gaunter '= 0.01572
Unsmoothed Count ' more precious ' =  1
Smoothed Count ' more precious '= 0.036778
Unsmoothed Count ' his motives ' =  1
Smoothed Count ' his motives '= 0.220195
As we can see, there is huge reduction in the number of counts of particular bigrams, When we do Add-one Smoothing.This happens because we have distributed the probability mass of existing bigrams to the bigrams w

In [48]:
from random import randint
sent = test[randint(0, len(test)-1)]    # Randomly picking the sentence from test data set
print ("Random sentence from the test =", sent)

prob = probability(sent, "add1")
prob =np.exp(prob)
# print("Probability ", prob)
perp = pow(prob, 1/float(len(sent))) * len(sent)
print("Perplexity ,add1 =",  perp )


prob = probability(sent, "gd")
prob =np.exp(prob)
perp = pow(prob, 1/float(len(sent))) * len(sent) 
print("Perplexity ,Good Turing =",  perp) 

Random sentence from the test = <s> i had almost overcome my pride so far as to go back to the agency and inquire whether the place was still open when i received this letter from the gentleman himself </s>
inside add1
Perplexity ,add1 = 76.9025407596
Perplexity ,Good Turing = 105.484384118
